In [2]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('rect1.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold the image to obtain a binary image (assuming the object is in white color)
_, thresholded = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize variables to store the largest rectangle/square found
largest_area = 0
largest_rectangle = None

# Iterate through the contours to find the largest rectangle/square
for contour in contours:
    # Approximate the contour as a polygon
    epsilon = 0.04 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
    # Check if the polygon has 4 corners (indicating a rectangle or square)
    if len(approx) == 4:
        # Calculate the area of the polygon
        area = cv2.contourArea(approx)
        
        # Check if it's the largest area found so far
        if area > largest_area:
            largest_area = area
            largest_rectangle = approx

# If a rectangle/square is found, draw it on the original image
if largest_rectangle is not None:
    cv2.drawContours(image, [largest_rectangle], -1, (0, 255, 0), 2)  # Draw a green rectangle

    # Calculate the perimeter of the rectangle/square
    perimeter = cv2.arcLength(largest_rectangle, True)

    # Calculate the centroid of the rectangle/square
    M = cv2.moments(largest_rectangle)
    centroid_x = int(M["m10"] / M["m00"])
    centroid_y = int(M["m01"] / M["m00"])

    # Display the perimeter and centroid
    print(f"Parameter of the object: {perimeter}")
    print(f"Centroid of the object (x, y): ({centroid_x}, {centroid_y})")

    # Show the image with the detected rectangle
    cv2.imshow('Image with Rectangle', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No rectangle/square found in the image.")


Parameter of the object: 654.0
Centroid of the object (x, y): (175, 136)


In [6]:
import cv2
import numpy as np

def determine_gender(image1_path, image2_path):
    # Load the images
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Perform color analysis (e.g., check for dominant colors)
    def analyze_colors(image):
        # Convert the image to the HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Define color ranges for red, blue, and pink (common colors associated with gender)
        lower_red = np.array([0, 100, 100])
        upper_red = np.array([10, 255, 255])
        lower_blue = np.array([100, 100, 100])
        upper_blue = np.array([140, 255, 255])
        lower_pink = np.array([140, 100, 100])
        upper_pink = np.array([170, 255, 255])

        # Create masks for each color range
        mask_red = cv2.inRange(hsv, lower_red, upper_red)
        mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
        mask_pink = cv2.inRange(hsv, lower_pink, upper_pink)

        # Count the number of pixels in each mask
        red_pixels = cv2.countNonZero(mask_red)
        blue_pixels = cv2.countNonZero(mask_blue)
        pink_pixels = cv2.countNonZero(mask_pink)

        return red_pixels, blue_pixels, pink_pixels

    red_pixels1, blue_pixels1, pink_pixels1 = analyze_colors(image1)
    red_pixels2, blue_pixels2, pink_pixels2 = analyze_colors(image2)

    # Define heuristics for gender determination
    # Calculate color dominance ratios
    dominance_ratio1 = red_pixels1 / max(1, blue_pixels1 + pink_pixels1)
    dominance_ratio2 = red_pixels2 / max(1, blue_pixels2 + pink_pixels2)

    if dominance_ratio1 > dominance_ratio2:
        return "Image 1 is a boy, Image 2 is a girl"
    elif dominance_ratio2 > dominance_ratio1:
        return "Image 1 is a girl, Image 2 is a boy"
    else:
        return "Both images have similar color characteristics. Unable to determine gender."

# Example usage:
image1_path = 'fig3.jpg'  # Replace with the path to your boy image
image2_path = 'fig4.jpg'  # Replace with the path to your girl image
result = determine_gender(image1_path, image2_path)
print(result)


Image 1 is a boy, Image 2 is a girl


In [ ]:
import cv2
import numpy as np

def determine_blurriness(original_path, blurred_path):
    # Load the original and blurred images
    original_image = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    blurred_image = cv2.imread(blurred_path, cv2.IMREAD_GRAYSCALE)

    # Calculate the Laplacian variance for each image
    original_laplacian_var = cv2.Laplacian(original_image, cv2.CV_64F).var()
    blurred_laplacian_var = cv2.Laplacian(blurred_image, cv2.CV_64F).var()

    # Define a threshold to distinguish between original and blurred images
    threshold = 1000  # Adjust this threshold as needed

    # Determine which image is blurred and which one is original
    if blurred_laplacian_var < threshold:
        return "Blurred Image", "Original Image"
    else:
        return "Original Image", "Blurred Image"

# Example usage:
original_image_path = 'fig5.jpg'  # Replace with the path to your original image
blurred_image_path = 'fig5_blur.jpg'    # Replace with the path to your blurred image

blurred_title, original_title = determine_blurriness(original_image_path, blurred_image_path)

# Display the images with titles
original_image = cv2.imread(original_image_path)
blurred_image = cv2.imread(blurred_image_path)

cv2.imshow(original_title, original_image)
cv2.imshow(blurred_title, blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:
import cv2
import numpy as np

def find_color_bar_areas(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to isolate the color bars
    thresholded = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize an empty list to store areas
    areas = []

    # Loop through each contour and calculate its area
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)

        # Find the centroid of the contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0

        # Draw the contour and label the area at the centroid with a very small font size
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
        cv2.putText(image, f"Area: {area:.2f} px^2", (cx - 40, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (0, 0, 255), 1)

        # Append the area to the list
        areas.append(area)

    # Display the image with contours and area labels
    cv2.imshow("Color Bars with Areas", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Return the areas of the color bars
    return areas

# Example usage:
image_path = 'fig1.jpg'  # Replace with the path to your image
areas = find_color_bar_areas(image_path)


In [35]:
import cv2
import numpy as np

def calculate_percentage_area(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the color ranges for each colored bar
    color_ranges = [
        {"name": "Yellow", "lower": (25, 50, 50), "upper": (35, 255, 255)},
        {"name": "Light Gray", "lower": (0, 0, 160), "upper": (90, 30, 255)},
        {"name": "Gray", "lower": (0, 0, 90), "upper": (90, 30, 160)},
        {"name": "Dark Gray", "lower": (0, 0, 0), "upper": (90, 30, 90)},
    ]

    # Create a mask for the red arrow
    lower_red = np.array([0, 0, 150])
    upper_red = np.array([50, 50, 255])
    mask_red = cv2.inRange(hsv_image, lower_red, upper_red)

    # Initialize a dictionary to store bar names and covered areas
    results = {}

    # Loop through each color range and find the corresponding bars
    for color_range in color_ranges:
        lower_color = np.array(color_range["lower"])
        upper_color = np.array(color_range["upper"])

        # Create a mask for the current color range
        mask_color = cv2.inRange(hsv_image, lower_color, upper_color)

        # Find contours in the mask
        contours_color, _ = cv2.findContours(mask_color, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize the total area of bars in this color range
        total_area = 0

        # Loop through each detected contour in the color range
        for contour in contours_color:
            # Calculate the area of the current bar
            bar_area = cv2.contourArea(contour)

            # Add the bar area to the total area
            total_area += bar_area

        # Calculate the percentage area covered by the red arrow
        if total_area > 0:
            percentage_covered = (total_area / np.sum(mask_red / 255)) * 100
        else:
            percentage_covered = 0

        # Store the results in the dictionary
        results[color_range["name"]] = percentage_covered

    return results

# Example usage:
image_path = 'fig2.jpg'  # Replace with the path to your image
percentage_areas = calculate_percentage_area(image_path)

# Display the results
print("Bar Area Covered(%):")
for color, percentage in percentage_areas.items():
    print(f"{color}: {percentage:.2f}%")


Bar Area Covered(%):
Yellow: 0.00%
Light Gray: 97.43%
Gray: 0.00%
Dark Gray: 0.00%


In [1]:
import cv2
import numpy as np

def segment_and_measure_parts(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to the HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the lower and upper HSV color thresholds for each part
    color_ranges = [
        {"name": "Part 1", "lower": (0, 0, 0), "upper": (179, 255, 60)},  # Define the HSV range for each part
        {"name": "Part 2", "lower": (0, 0, 61), "upper": (179, 255, 120)},
        {"name": "Part 3", "lower": (0, 0, 121), "upper": (179, 255, 180)},
        {"name": "Part 4", "lower": (0, 0, 181), "upper": (179, 255, 240)},
        {"name": "Part 5", "lower": (0, 0, 241), "upper": (179, 255, 255)},
    ]

    # Initialize a list to store measurements
    measurements = []

    # Loop through each defined color range
    for color_range in color_ranges:
        lower_color = np.array(color_range["lower"], dtype=np.uint8)
        upper_color = np.array(color_range["upper"], dtype=np.uint8)

        # Create a mask for the current color range
        mask = cv2.inRange(hsv_image, lower_color, upper_color)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize maximum width and height
        max_width = 0
        max_height = 0

        # Loop through each detected contour in the color range
        for contour in contours:
            # Find the bounding box of the contour
            x, y, w, h = cv2.boundingRect(contour)

            # Update maximum width and height
            max_width = max(max_width, w)
            max_height = max(max_height, h)

            # Draw a rectangle around the part (for visualization)
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Append measurements to the list
        measurements.append((color_range["name"], max_width, max_height))

    # Display the image with bounding boxes
    cv2.imshow('Finger Parts with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Display maximum width and height for each part
    print("Part Measurements:")
    print("Part\tMax Width\tMax Height")
    for measurement in measurements:
        part, max_width, max_height = measurement
        print(f"{part}\t{max_width}\t\t{max_height}")

# Example usage:
image_path = 'finger-bones.jpg'  # Replace with the path to your image
segment_and_measure_parts(image_path)


Part Measurements:
Part	Max Width	Max Height
Part 1	36		35
Part 2	13		20
Part 3	36		93
Part 4	41		124
Part 5	96		342
